# ✔ Introduction

Waste has been detrimental to the environment's health and its improper disposal has caused serious problems to our health as well as our surroundings.

Recycling is the process of converting waste materials into new materials and objects.It can also reduce the waste of potentially useful materials and the consumption of fresh raw materials, lowering energy consumption, air pollution (from incineration), and water pollution (from landfilling). Many types of glass, paper, cardboard, metal, plastic, tyres, textiles, batteries, and electronics are recyclable. Composting and other biodegradable waste reuse, such as food and garden waste, is also a type of recycling. Recycling materials are either delivered to a household recycling centre or collected from curbside bins, where they are sorted, cleaned, and reprocessed into new materials for the manufacture of new products.
Hence separating recyclable and organic waste is the first step towards systematic waste disposal.

 **In this notebook, we will use a Convolutional Neural Network(CNN) with Transfer Learning to classify waste as organic or recyclable.**

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2

import warnings
warnings.filterwarnings('ignore')
import os
for dirname, _, _ in os.walk('/kaggle/input'):
        print(dirname)

In [2]:
train_path = "/kaggle/input/waste-classification-data/DATASET/TRAIN/"
test_path = "/kaggle/input/waste-classification-data/DATASET/TEST/"

In [3]:
from tensorflow.keras import backend as K
!pip install --upgrade tensorflow-model-optimization
import tensorflow_model_optimization as tfmot

## Import Libraries

In [27]:
import tensorflow as tf
from tensorflow.keras import models, layers


In [4]:

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.utils import plot_model
from glob import glob
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.xception import Xception


## Visualization

In [6]:
x_data = [] 
y_data = [] 

for category in glob(train_path+'/*'):
    for file in tqdm(glob(category+'/*')):
        img_array=cv2.imread(file)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        x_data.append(img_array) 
        y_data.append(category.split("/")[-1])
        
data=pd.DataFrame({'image': x_data,'label': y_data})

In [7]:
data.shape

In [8]:
from collections import Counter
Counter(y_data)


In [9]:
colors = ['#FEC8D8','#D9F4E0']
plt.pie(data.label.value_counts(),startangle=90,explode=[0.05,0.05],autopct='%0.2f%%',
        labels=['Organic', 'Recyclable'], colors= colors,radius=2)
plt.show()

In [10]:
plt.figure(figsize=(20,15))
for i in range(9):
    plt.subplot(4,3,(i%12)+1)
    index=np.random.randint(15000)
    plt.title('This image is of {0}'.format(data.label[index]),fontdict={'size':20,'weight':'bold'})
    plt.imshow(data.image[index])
    plt.tight_layout()

<div style="color:black;
           display:fill;
           border-radius:5px;
           background-color:#FAF4ED;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">

<p style="padding: 10px;
              color:black;">
O -> Organic
R -> Recyclable
</p>
</div>

In [11]:
className = glob(train_path + '/*' )
numberOfClass = len(className)
print("Number Of Class: ",numberOfClass)

# Convolutional Neural Network - CNN

In [12]:
input_size = 244
datagenerator = ImageDataGenerator(
    rescale=1.0 / 255, horizontal_flip=True, vertical_flip=True, zoom_range=0.2, shear_range=0.2, rotation_range = 10,validation_split=0.2
)
# Train data

train_generator = datagenerator.flow_from_directory(
    directory=train_path, target_size=(input_size, input_size), class_mode="binary", batch_size=128,subset="training"
)

valid_generator = datagenerator.flow_from_directory(
    directory=train_path, target_size=(input_size, input_size), class_mode="binary", batch_size=128,subset="validation"
)

# Test data
test_generator = datagenerator.flow_from_directory(
    directory=test_path, target_size=(input_size, input_size), class_mode="binary", batch_size=128
)

In [16]:
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(input_size, input_size, 3))

for layer in base_model.layers:
    layer.trainable = False

model=Sequential()
model.add(base_model)
model.add(Dropout(0.2))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(1024,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(1024,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
# Compiling the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [17]:
# Summary

model.summary()

In [ ]:
plot_model(model)

In [18]:
early_stopping_val_loss_callback = EarlyStopping(monitor='val_loss', patience=5)
early_stopping_loss_callback = EarlyStopping(monitor='loss', patience=5)

checkpoint_callback = ModelCheckpoint('model_MobileNetV2.h5', monitor='accuracy', verbose=1, save_best_only=True, mode='auto')

model_history = model.fit(
    train_generator,
    epochs=15,
    validation_data=valid_generator,
    callbacks=[early_stopping_val_loss_callback, early_stopping_loss_callback, checkpoint_callback]
)

In [19]:
plt.figure(figsize=[10,6])
plt.plot(model_history.history["accuracy"], label = "Train acc")
plt.plot(model_history.history["val_accuracy"], label = "Validation acc")
plt.legend()
plt.show()

In [20]:
plt.figure(figsize=(10,6))
plt.plot(model_history.history['loss'], label = "Train loss")
plt.plot(model_history.history['val_loss'], label = "Validation loss")
plt.legend()
plt.show()

In [21]:
model_loss, model_acc = model.evaluate(test_generator)
print("Model has a loss of %.2f and accuracy %.2f%%" % (model_loss, model_acc*100))

## Post Training Quantization

In [22]:
def apply_quantization(layer):
    if (
        isinstance(layer, layers.Dense)
        or isinstance(layer, layers.MaxPool2D)
        or isinstance(layer, layers.Conv2D)
    ):
        return tfmot.quantization.keras.quantize_annotate_layer(layer)
    return layer

In [28]:
annotated_model = tf.keras.models.clone_model(
    model,
    clone_function=apply_quantization
)

quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)
quant_aware_model.summary()

In [32]:
quant_aware_model.compile(
    optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
)

In [ ]:
q_history = quant_aware_model.fit(train_generator,
    epochs=15,
    validation_data=valid_generator
)

In [ ]:
quant_model_loss, quant_model_acc = quant_aware_model.evaluate(test_generator)
print("Quantized Model has a loss of %.2f and accuracy %.2f%%" % (quant_model_loss, quant_model_acc*100))

## Converting Quanitization Aware Model to TF Lite Model

In [35]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

In [46]:
type(test_generator)

In [49]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
  test_path,
  seed=123,
  image_size=(input_size, input_size),
  batch_size=128
)

In [ ]:

def evaluate_tflite_model(dataset, interpreter):
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    prediction_digits = []
    test_labels = []
    for image, label in dataset.unbatch().take(dataset.unbatch().cardinality()):

        test_image = np.expand_dims(image, axis=0).astype(np.float32)
        interpreter.set_tensor(input_index, test_image)
        interpreter.invoke()
        
        output = interpreter.tensor(output_index)
        digit = np.argmax(output()[0])
        prediction_digits.append(digit)
        test_labels.append(label)

    prediction_digits = np.array(prediction_digits)
    accuracy = (prediction_digits == test_labels).mean()
    return accuracy

interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_tflite_model(dataset, interpreter)

print('Quant TFLite test_accuracy:', test_accuracy)

In [55]:
import os
model_version = 1

with open(
    f"tf-lite-model{model_version}.tflite",
    'wb'
) as f:
    f.write(quantized_tflite_model)

## Testing with examples

In [63]:
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]
class_names = ['organic','recyclable']
plt.figure(figsize=(15, 15))
for images, labels in dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        actual_class = class_names[labels[i]]

        test_image = np.expand_dims(images[i], axis=0).astype(np.float32)
        interpreter.set_tensor(input_index, test_image)
        interpreter.invoke()
        output = interpreter.tensor(output_index)
        pred = output()[0][0]
        if pred > 0.5:
            confidence = np.round(pred*100, decimals=2)
            predicted_class = 'recyclable'
        else:
            confidence = np.round((1-pred)*100, decimals=2)
            predicted_class = 'organic'
        plt.title(f"Actual: {actual_class}\n Predicted Class: {predicted_class}\n Confidence: {confidence}%")
        plt.axis('off')


In [60]:
file1 = test_path + 'O/O_12568.jpg'
file = test_path + 'R/R_10000.jpg'
img_array=cv2.imread(file1)
img=cv2.imread(file)

img_array = cv2.resize(img_array, (input_size, input_size))
img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
img_array = np.reshape(img_array, [1, input_size, input_size, 3]) / 255.0

pred = float(model(img_array).numpy()[0])
if pred > 0.5:
    pred = np.round(pred*100, decimals=2)
else:
    pred = np.round((1-pred)*100, decimals=2)
print(pred)